In [60]:
import re
# nltk?
from english_words import get_english_words_set

wl = 5  # word length
# getting all wl-len words for wordle
words = [word for word in get_english_words_set(['web2'], lower=True) if len(word) == wl]
print("found", len(words), "words")
words = ' '.join(words)

found 9979 words


In [87]:
def match_words(to_find=".....", bad_c="", good_c="", guesses=[], verbose=False):
    """ gets a list of all wl-letter words given criteria 
        to_find => the word matches these letters in this order with '.' for unknown
        bad_c => the word doesnt contain these characters
        good_c => the word does contain these characters"""
    # get all words that match this sequence
    found_words = re.findall(to_find, words)
#     if verbose: 
#         print("Found words:", found_words)

    # character subset
    bad_c += ' '  # addition for space delim in regex
    res = []
    for word in found_words:
        can_add = True
        for c in bad_c:  # subtract words with bad characters
            if c in word:
                can_add = False
        if can_add:
            for c in good_c:  # add words with good characters
                if c not in word:
                    can_add = False
        if can_add and word in guesseS:
            can_add = False:
        if can_add:
            res.append(word)
    if verbose:
        print("\nResulting words:")
        print(res)
    return res

potential_words = match_words(".....", "arise", "", [], True)  # easiest

SyntaxError: invalid syntax (2969847129.py, line 24)

In [83]:
def best_play(good_words, verbose=False):
    """ returns the best play for wordle based on character frequency 
    Note: this is not necessarily optimal since word difficulty needs to be taken into account
     (i.e. 'later' is better then 'artie' because nobody gona put artie in wordle XD)"""
    freq = {}  # get frequency map of all characters
    freq2 = [{} for i in range(wl)]   # get frequency map of character frequency in each position
    for word in good_words:  # TODO: weight freq by how common the word is
        for c in set(word): # save raw freq
            freq[c] = freq.get(c, 0) + 1
        for i, c in enumerate(word):  # save freq for specific places
            freq2[i][c] = freq2[i].get(c, 0) + 1
    if verbose:
        print("*Frequency Map")  # show in sorted order
        print({k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)})
        for i in range(len(freq2)):
            print("Frequency2 Map", i)  # show in sorted order
            print({k: v for k, v in sorted(freq2[i].items(), key=lambda item: item[1], reverse=True)})
    
    score = {}  # get score of each word
    for word in good_words:
        sc = 0
        for c in set(word):  # add freq of letter occurring in any word
            sc += freq[c]  # aka chance of -
        for i, c in enumerate(word):  # add freq of letter occurring in that spot in any word (half weighted)
            sc += freq2[i].get(c, 0) # aka chance of * 
            
        score[word] = sc  # TODO: account for previously guessed words
    # sort possible words based on score
    res = sorted(good_words, key=lambda word: score[word], reverse=True)
    if verbose:
        print("\nWords sorted by best option:")
        print(res)
    return res

play = best_play(words.split(" "), True)[0]

*Frequency Map
{'a': 4798, 'e': 4123, 'r': 3270, 'i': 3053, 'o': 2877, 's': 2608, 'l': 2514, 'n': 2500, 't': 2451, 'u': 2176, 'y': 1755, 'c': 1689, 'd': 1542, 'h': 1506, 'm': 1498, 'p': 1359, 'b': 1233, 'g': 1192, 'k': 1017, 'w': 735, 'f': 647, 'v': 523, 'z': 281, 'x': 232, 'j': 231, 'q': 98}
Frequency2 Map 0
{'s': 1267, 'a': 788, 'c': 732, 'b': 713, 't': 666, 'p': 579, 'm': 518, 'd': 451, 'g': 451, 'r': 436, 'f': 397, 'l': 396, 'h': 307, 'w': 294, 'k': 272, 'u': 270, 'n': 254, 'e': 245, 'o': 210, 'i': 178, 'v': 164, 'j': 156, 'y': 87, 'z': 66, 'q': 60, 'x': 22}
Frequency2 Map 1
{'a': 1752, 'o': 1314, 'e': 1199, 'i': 959, 'u': 867, 'r': 785, 'l': 585, 'h': 534, 'n': 389, 't': 234, 'p': 197, 'c': 172, 'y': 153, 'm': 151, 'w': 134, 's': 110, 'd': 91, 'k': 65, 'b': 65, 'g': 62, 'v': 46, 'x': 39, 'z': 27, 'f': 23, 'q': 14, 'j': 12}
Frequency2 Map 2
{'a': 982, 'r': 957, 'i': 825, 'o': 766, 'n': 752, 'e': 636, 'l': 609, 'u': 538, 't': 502, 's': 410, 'm': 394, 'd': 350, 'b': 303, 'g': 296, 'c

In [77]:
def play_wordle(target, guess):
    """ executes 1 round of wordle and returns feedback """
    assert len(target) == wl and len(guess) == wl
    res = ""
    for i in range(len(target)):
        if target[i] == guess[i]:
            res += '*'
        elif guess[i] in target:
            res += '-'
        else:
            res += 'x'
    return res

def wordle(target, max_guess=6):
    """ executes max_guess rounds of wordle and tells you if you won :) 
    (returns all your guesses & feedback) """
    i = 0
    results = []
    while i <= max_guess:
        inword = str(input("Your Guess: ")).lower().strip()
        feedback = play_wordle(target, inword)
        results.append((inword, feedback))
        print("Feedback: ", feedback)
        print()
        if feedback == '*****':
            print("Congrats! You win :)\n")
            return results
        i += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return results
wordle("gummy")

Your Guess: dream
Feedback:  xxxx-

Your Guess: moist
Feedback:  -xxxx

Your Guess: gummy
Feedback:  *****

Congrats! You win :)


[('dream', 'xxxx-'), ('moist', '-xxxx'), ('gummy', '*****')]

In [86]:
def bot_wordle(target, num_guesses=6, verbose=False):
    """ shows wordle guesses & results for a freq-optimal bot
    Note: its not very good for reasons discussed earlier """
    counter = 1
    potential_words = words.split(' ')
    guessed = []
    while counter <= num_guesses:
        inword = best_play(potential_words)[0]
        print("Guessed:", inword)
        
        feedback = play_wordle(target, inword)
        print("Feedback:", feedback)
        
        if feedback == '*****':
            print("Congrats! You win :)")
            return
        else:  # find next best option
            to_find = ""
            good_c = ""
            bad_c = ""
            for i, c in enumerate(feedback):  # update params based on feedback
                if c == "*":
                    to_find += inword[i]
                else:
                    to_find += '.'
                    if c == "-":
                        good_c += inword[i]
                    else:
                        bad_c += inword[i]
            # update words for next round
            if verbose: print("Update:", ', '.join([to_find, bad_c, good_c]))
            potential_words = match_words(to_find, bad_c, good_c, guesses)
        print()
        counter += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return

bot_wordle("bravo", 6, True)

# TODO: make it so bot looks at previous guesses and knows - letters can be in that position*

Guessed: tarie
Feedback: x--xx
Update: ....., tie, ar

Guessed: carob
Feedback: x----
Update: ....., c, arob

Guessed: boran
Feedback: *---x
Update: b...., n, ora

Guessed: boral
Feedback: *---x
Update: b...., l, ora

Guessed: borax
Feedback: *---x
Update: b...., x, ora

Guessed: bolar
Feedback: *-x--
Update: b...., l, oar

Sorry thats not right...
The correct word was: bravo
